In [2]:
import torch

In [3]:
new = torch.empty(1,1,1,6)

In [4]:
new.mT.shape, new.T.shape   ## T is transpose & mT is transpose of last 2 dims

<ipython-input-4-4c93d4e00b17>:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3637.)
  new.mT.shape, new.T.shape   ## T is transpose & mT is transpose of last 2 dims


(torch.Size([1, 1, 6, 1]), torch.Size([6, 1, 1, 1]))

In [5]:
torch.rand(2,3).H.shape  ## conjugate and transpose of a 2-D matrix, has a mH similar to mT

torch.Size([3, 2])

In [6]:
new.transpose(-1,-1).shape

torch.Size([1, 1, 1, 6])

In [7]:
w = 0.7
b = 0.3


start = 0
end =  1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = w * X + b

In [8]:
X.shape, y.shape

(torch.Size([50, 1]), torch.Size([50, 1]))

In [9]:
seventy = int(0.7 * len(X))
X_train, y_train, X_test, y_test = X[:seventy], y[:seventy], X[seventy:], y[seventy:]
X_train.shape,y_train.shape, X_test.shape, y_test.shape

(torch.Size([35, 1]),
 torch.Size([35, 1]),
 torch.Size([15, 1]),
 torch.Size([15, 1]))

In [10]:
from torch import nn

In [11]:
## boiler plate for all neural network model class
## forward method needs to be overwritten to specify computation for every iteration

class LinRegModel(nn.Module):
  def __init__(self) -> None:
    super().__init__()
    self.weights = nn.Parameter(torch.randn(1,
                                requires_grad=True,
                                dtype=torch.float)
                                )
    self.bias = nn.Parameter(torch.randn(1,
                                         requires_grad=True,
                                         dtype=torch.float)
                                )

  def forward(self, x:torch.tensor) -> torch.Tensor:
    return self.weights * x + self.bias   ## linear reg



In [12]:
torch.manual_seed(43)
my_model = LinRegModel()

In [13]:
list(my_model.parameters())

[Parameter containing:
 tensor([-0.6484], requires_grad=True),
 Parameter containing:
 tensor([-0.7058], requires_grad=True)]


### checkout utils.dataset and dataloader

nn.Module -- contains the larger building blocks (layers)

nn.Parameter -- contains the smaller parameters like weights and biases (put these together to make nn.Module(s))

forward() -- tells the larger blocks how to make calculations on inputs (tensors full of data) within nn.Module(s)

torch.optim -- contains optimization methods on how to improve the parameters within nn.Parameter to better represent input data

In [14]:
my_model.state_dict()

OrderedDict([('weights', tensor([-0.6484])), ('bias', tensor([-0.7058]))])

In [15]:
y_preds = my_model(X_test)
y_preds, y_test

(tensor([[-1.1597],
         [-1.1727],
         [-1.1857],
         [-1.1986],
         [-1.2116],
         [-1.2246],
         [-1.2375],
         [-1.2505],
         [-1.2635],
         [-1.2764],
         [-1.2894],
         [-1.3024],
         [-1.3153],
         [-1.3283],
         [-1.3413]], grad_fn=<AddBackward0>),
 tensor([[0.7900],
         [0.8040],
         [0.8180],
         [0.8320],
         [0.8460],
         [0.8600],
         [0.8740],
         [0.8880],
         [0.9020],
         [0.9160],
         [0.9300],
         [0.9440],
         [0.9580],
         [0.9720],
         [0.9860]]))

In [16]:
with torch.inference_mode():   ## removes gradient track ## replaces torch.no_grad() from lder pytorch
  y_pred = my_model(X_test)

y_pred

tensor([[-1.1597],
        [-1.1727],
        [-1.1857],
        [-1.1986],
        [-1.2116],
        [-1.2246],
        [-1.2375],
        [-1.2505],
        [-1.2635],
        [-1.2764],
        [-1.2894],
        [-1.3024],
        [-1.3153],
        [-1.3283],
        [-1.3413]])

basic scoring using loss functions like mse_loss and l1_loss ,
l1_loss is MAE

In [17]:
%%time
torch.nn.functional.l1_loss(y_pred, y_test), torch.nn.functional.mse_loss(y_pred, y_test)

CPU times: user 43 µs, sys: 840 µs, total: 883 µs
Wall time: 2.26 ms


(tensor(2.1385), tensor(4.5868))

In [18]:
%%time
## Mean of absoulte of element wise difference
l=[]
for i,j in zip(y_pred, y_test):
  l.append(abs(i-j))

diff = torch.tensor(l)
diff.mean()

CPU times: user 651 µs, sys: 5 µs, total: 656 µs
Wall time: 665 µs


tensor(2.1385)

In [19]:
loss_func = nn.L1Loss()

optimiser = torch.optim.SGD(params = my_model.parameters(),
                            lr = 0.001
                            )

For training loop, we need

- loop through data
- foward propogation using forward method
- calulate loss
- optimise zero grad
- loss backward - move backward in network to calculate the gradients of each of the params with respect to loss
- optimiser step to try to reduce loss, ie, gradient descent

In [20]:
%%time
epochs = 10000

for epoch in range(epochs):
  my_model.train()
  y_pred = my_model(X_train)
  loss = loss_func(y_train, y_pred)
  optimiser.zero_grad()   ## required if multiple epochs, check out docs
  loss.backward()
  optimiser.step()
  my_model.eval()

  with torch.inference_mode():
    y_pred_test = my_model(X_test)
    test_loss = loss_func(y_test, y_pred_test)


  if (epochs - epoch) <= 10:
    print(f"epoch {epoch}, loss {loss}, test_loss {test_loss}")

epoch 9990, loss 0.0006866072071716189, test_loss 0.000955816125497222
epoch 9991, loss 0.0004289737844374031, test_loss 0.00032976866350509226
epoch 9992, loss 0.0006866072071716189, test_loss 0.000955816125497222
epoch 9993, loss 0.0004289737844374031, test_loss 0.00032976866350509226
epoch 9994, loss 0.0006866072071716189, test_loss 0.000955816125497222
epoch 9995, loss 0.0004289737844374031, test_loss 0.00032976866350509226
epoch 9996, loss 0.0006866072071716189, test_loss 0.000955816125497222
epoch 9997, loss 0.0004289737844374031, test_loss 0.00032976866350509226
epoch 9998, loss 0.0006866072071716189, test_loss 0.000955816125497222
epoch 9999, loss 0.0004289737844374031, test_loss 0.00032976866350509226
CPU times: user 5.78 s, sys: 109 ms, total: 5.89 s
Wall time: 5.94 s


In [21]:
list(my_model.parameters())

[Parameter containing:
 tensor([0.6993], requires_grad=True),
 Parameter containing:
 tensor([0.3009], requires_grad=True)]

In [22]:
with torch.inference_mode():
  y_pred = my_model(X_test)

In [23]:
torch.nn.functional.l1_loss(y_pred, y_test), torch.nn.functional.mse_loss(y_pred, y_test)

(tensor(0.0003), tensor(1.1255e-07))

current timestamp - 7:40:00

In [36]:
torch.save(obj = my_model.state_dict(), f='first_lin_regressor.pth')   ## asve onj as state dict instead of saving entire model, check ddocs